In [1]:
import pandas as pd
import numpy as np
import os
import json
import pickle
from collections import defaultdict
from datetime import datetime
import openai
import requests
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored
openai.api_key = os.environ.get('OPENAI_API_KEY')

In [2]:
user_test_path = '../fashion_data/processed_data/user_test.json'
user_valid_path = '../fashion_data/processed_data/user_valid.json'
user_train_path = '../fashion_data/processed_data/user_train.json'

In [3]:
with open(user_test_path, 'rb') as f:
    user_test_dict = json.load(f)
print(len(user_test_dict))

1273


In [4]:
with open(user_valid_path, 'rb') as f:
    user_valid_dict = json.load(f)
print(len(user_valid_dict))

1273


In [5]:
with open(user_train_path, 'rb') as f:
    user_train_dict = json.load(f)
print(len(user_train_dict))

1273


In [6]:
user_id = '1'
print(user_train_dict[user_id])
print(user_valid_dict[user_id])
print(user_test_dict[user_id])

[11, 12, 13, 14, 15]
[12, 13, 14, 15, 16]
[13, 14, 15, 16, 17]


In [7]:
len(user_train_dict[user_id])

5

In [8]:
item_information_path = '../fashion_data/processed_data/processed_item_information.json'
with open(item_information_path, 'r') as f:
    item_information_dict = json.load(f)
print(len(item_information_dict))

6089


In [9]:
print(item_information_dict['0'])

{'title': 'Allegra K Lady Long Sleeve Letter Pattern Pullover Knit Shirt Black S', 'description': '\n\nThe product is a cute and comfy t-shirt, fitting well with leggings, and available in various sizes. It has a good quality fabric and length, but is very casual and simple.\n\nNote: The product description is written in 25 words as requested, and it highlights both'}


In [10]:
ratings_path = '../fashion_data/processed_data/ratings.csv'
ratings_df = pd.read_csv(ratings_path)
ratings_df.head()

,item,user,rating,timestamp
0,0,0,4.0,1452816000
1,1,0,4.0,1453766400
2,2,0,3.0,1453766400
3,3,0,4.0,1453766400
4,4,0,4.0,1453766400


In [11]:
# def get_item_description(item_information):
#     item_details = ""
#     if 'title' in item_information:
#         item_details += item_information['title']
#     if 'brand' in item_information:
#         item_details += f" Brand: {item_information['brand']}"
#     if 'price' in item_information:
#         item_details += f" Price: {item_information['price']}"
#     if 'description' in item_information:
#         item_details += f" Description: {item_information['description']}"
#     return item_details

In [18]:
train_data = []
instruction = "Given the user preference and unpreference, identify whether the user will like the target item by answering \"Yes.\" or \"No.\"."
# print(instruction)
cnt = 0
for user, item_list in user_train_dict.items():  
    # print(user, item_list)
    user_rating_dict = ratings_df[ratings_df['user'] == int(user)]
    user_dict = dict()
    user_dict["instruction"] = instruction
    cnt += 1
    liked_items = []
    disliked_items = []

    for item in item_list[:-1]:
        user_item_rating = user_rating_dict[user_rating_dict['item'] == item]['rating'].values[0]
        if user_item_rating > 3:
            liked_items.append(item)
        else:
            disliked_items.append(item)
    
    input = ""
    if len(liked_items) > 0:
        input += "\nUser Preference:"
        for item in liked_items:
            item_title = item_information_dict[str(item)]['title']
            input += f"\"{item_title}\", "
    if len(disliked_items) > 0:
        input += "\nUser Unpreference:"
        for item in liked_items:
            item_title = item_information_dict[str(item)]['title']
            input += f"\"{item_title}\", "
    
    target_item = item_information_dict[str(item_list[-1])]['title']
    input += f"\n Whether the user will like the target movie \"{target_item}\"?"
    # print(input)
    user_dict["input"] = input
    if user_rating_dict[user_rating_dict['item'] == item_list[-1]]['rating'].values[0] > 3:
        user_dict["output"] = "Yes."
    else:
        user_dict["output"] = "No."
    train_data.append(user_dict)

In [19]:
print(len(train_data))

1273


In [20]:
train_data[0]

{'instruction': 'Given the user preference and unpreference, identify whether the user will like the target item by answering "Yes." or "No.".',
 'input': '\nUser Preference:"Allegra K Women Boat Neck Batwing Sleeves Pullover Tops Fuchsia M", "Allegra K Woman Boat Neck Button Decor Front Batwing Tunic Top XL Dark Blue", "Allegra K Women Sleeveless Sheer Mesh Yoke Mini Skater Dress L Black", \nUser Unpreference:"Allegra K Women Boat Neck Batwing Sleeves Pullover Tops Fuchsia M", "Allegra K Woman Boat Neck Button Decor Front Batwing Tunic Top XL Dark Blue", "Allegra K Women Sleeveless Sheer Mesh Yoke Mini Skater Dress L Black", \n Whether the user will like the target movie "DJT Cowl Neck Button Embellished Ruched Long Sleeve Blouse Top Grey XL"?',
 'output': 'No.'}

In [21]:
total_len = 0
for data in train_data:
    total_len += len(data['input'].split(' '))
print(total_len/len(train_data))

897.6928515318147


In [22]:
# cnt = 0
cleaned_train_data = []
for data in train_data:
    if len(data['input'].split(' ')) <= 120:
        cleaned_train_data.append(data)
print(len(cleaned_train_data))

967


In [23]:
total_len = 0
for data in train_data:
    total_len += len(data['input'])
print(total_len/len(train_data))

14505.81775333857


In [24]:
total_len = 0
for data in cleaned_train_data:
    total_len += len(data['input'])
print(total_len/len(cleaned_train_data))

462.8200620475698


In [16]:
valid_data = []
instruction = "Given the user preference and unpreference, identify whether the user will like the target item by answering \"Yes.\" or \"No.\"."
# print(instruction)
cnt = 0
for user, item_list in user_valid_dict.items():  
    # print(user, item_list)
    user_rating_dict = ratings_df[ratings_df['user'] == int(user)]
    user_dict = dict()
    user_dict["instruction"] = instruction
    cnt += 1
    liked_items = []
    disliked_items = []

    for item in item_list[:-1]:
        user_item_rating = user_rating_dict[user_rating_dict['item'] == item]['rating'].values[0]
        if user_item_rating > 3:
            liked_items.append(item)
        else:
            disliked_items.append(item)
    
    input = ""
    if len(liked_items) > 0:
        input += "\nUser Preference:"
        for item in liked_items:
            item_title = item_information_dict[str(item)]['title']
            input += f"\"{item_title}\", "
    if len(disliked_items) > 0:
        input += "\nUser Unpreference:"
        for item in liked_items:
            item_title = item_information_dict[str(item)]['title']
            input += f"\"{item_title}\", "
    
    target_item = item_information_dict[str(item_list[-1])]['title']
    input += f"\n Whether the user will like the target movie \"{target_item}\"?"
    # print(input)
    user_dict["input"] = input
    if user_rating_dict[user_rating_dict['item'] == item_list[-1]]['rating'].values[0] > 3:
        user_dict["output"] = "Yes."
    else:
        user_dict["output"] = "No."
    valid_data.append(user_dict)

In [23]:
total_len = 0
for data in valid_data:
    total_len += len(data['input'].split(' '))
print(total_len/len(valid_data))

862.6637863315004


In [24]:
cleaned_valid_data = []
for data in valid_data:
    if len(data['input'].split(' ')) <= 120:
        cleaned_valid_data.append(data)
print(len(cleaned_valid_data))

980


In [25]:
print(cleaned_train_data[1])
print(cleaned_valid_data[1])

{'instruction': 'Given the user preference and unpreference, identify whether the user will like the target item by answering "Yes." or "No.".', 'input': '\nUser Preference:"Jastore Little Girls\' Clothing Set Sleeveless Shirt+ Pants with Belt (3-4 Years)", "ANGVNS Women Casual Long Sleeve Slim Fit Party Bodycon Sheath Dress 2017", "FINEJO Women Chiffon Round Neck Sleeveless Maxi Dress, Green M", "TXZTrade Women\'s Floral Tankini Swimdress Cover Up Vintage Two-Piece Swimsuit with Swimming trunks TXZ9019-Blue-XL", \n Whether the user will like the target movie "C2U Women Pullover Elegant Wave Flouncing Long Sleeve Chiffon Shirt,X-Large,Purple"?', 'output': 'Yes.'}
{'instruction': 'Given the user preference and unpreference, identify whether the user will like the target item by answering "Yes." or "No.".', 'input': '\nUser Preference:"ANGVNS Women Casual Long Sleeve Slim Fit Party Bodycon Sheath Dress 2017", "FINEJO Women Chiffon Round Neck Sleeveless Maxi Dress, Green M", "TXZTrade Wom

In [12]:
test_data = []
instruction = "Given the user preference and unpreference, identify whether the user will like the target item by answering \"Yes.\" or \"No.\"."
# print(instruction)
cnt = 0
for user, item_list in user_test_dict.items():  
    # print(user, item_list)
    user_rating_dict = ratings_df[ratings_df['user'] == int(user)]
    user_dict = dict()
    user_dict["instruction"] = instruction
    cnt += 1
    liked_items = []
    disliked_items = []

    for item in item_list[:-1]:
        user_item_rating = user_rating_dict[user_rating_dict['item'] == item]['rating'].values[0]
        if user_item_rating > 3:
            liked_items.append(item)
        else:
            disliked_items.append(item)
    
    input = ""
    if len(liked_items) > 0:
        input += "\nUser Preference:"
        for item in liked_items:
            item_title = item_information_dict[str(item)]['title']
            input += f"\"{item_title}\", "
    if len(disliked_items) > 0:
        input += "\nUser Unpreference:"
        for item in liked_items:
            item_title = item_information_dict[str(item)]['title']
            input += f"\"{item_title}\", "
    
    target_item = item_information_dict[str(item_list[-1])]['title']
    input += f"\n Whether the user will like the target movie \"{target_item}\"?"
    # print(input)
    user_dict["input"] = input
    if user_rating_dict[user_rating_dict['item'] == item_list[-1]]['rating'].values[0] > 3:
        user_dict["output"] = "Yes."
    else:
        user_dict["output"] = "No."
    test_data.append(user_dict)

In [33]:
total_len = 0
for data in train_data:
    total_len += len(data['input'].split(' '))
print(total_len/len(train_data))

897.6928515318147


In [34]:
print(len(test_data))
cleaned_test_data = []
for data in test_data:
    if len(data['input'].split(' ')) <= 120:
        cleaned_test_data.append(data)
print(len(cleaned_test_data))

1273
1249


In [28]:
print(len(cleaned_train_data))
# print(len(cleaned_valid_data))
print(len(cleaned_test_data))

967
518


In [29]:
print(cleaned_train_data[0])
print(cleaned_valid_data[0])
print(cleaned_test_data[0])

{'instruction': 'Given the user preference and unpreference, identify whether the user will like the target item by answering "Yes." or "No.".', 'input': '\nUser Preference:"Allegra K Women Boat Neck Batwing Sleeves Pullover Tops Fuchsia M", "Allegra K Woman Boat Neck Button Decor Front Batwing Tunic Top XL Dark Blue", "Allegra K Women Sleeveless Sheer Mesh Yoke Mini Skater Dress L Black", \nUser Unpreference:"Allegra K Women Boat Neck Batwing Sleeves Pullover Tops Fuchsia M", "Allegra K Woman Boat Neck Button Decor Front Batwing Tunic Top XL Dark Blue", "Allegra K Women Sleeveless Sheer Mesh Yoke Mini Skater Dress L Black", \n Whether the user will like the target movie "DJT Cowl Neck Button Embellished Ruched Long Sleeve Blouse Top Grey XL"?', 'output': 'No.'}
{'instruction': 'Given the user preference and unpreference, identify whether the user will like the target item by answering "Yes." or "No.".', 'input': '\nUser Preference:"Allegra K Woman Boat Neck Button Decor Front Batwing 

In [30]:
train_valid_data = cleaned_train_data.copy()
train_valid_data.extend(cleaned_valid_data)
print(len(train_valid_data), train_valid_data[:3])
print(len(train_data), len(valid_data), len(cleaned_test_data))

1947 [{'instruction': 'Given the user preference and unpreference, identify whether the user will like the target item by answering "Yes." or "No.".', 'input': '\nUser Preference:"Allegra K Women Boat Neck Batwing Sleeves Pullover Tops Fuchsia M", "Allegra K Woman Boat Neck Button Decor Front Batwing Tunic Top XL Dark Blue", "Allegra K Women Sleeveless Sheer Mesh Yoke Mini Skater Dress L Black", \nUser Unpreference:"Allegra K Women Boat Neck Batwing Sleeves Pullover Tops Fuchsia M", "Allegra K Woman Boat Neck Button Decor Front Batwing Tunic Top XL Dark Blue", "Allegra K Women Sleeveless Sheer Mesh Yoke Mini Skater Dress L Black", \n Whether the user will like the target movie "DJT Cowl Neck Button Embellished Ruched Long Sleeve Blouse Top Grey XL"?', 'output': 'No.'}, {'instruction': 'Given the user preference and unpreference, identify whether the user will like the target item by answering "Yes." or "No.".', 'input': '\nUser Preference:"Jastore Little Girls\' Clothing Set Sleeveless

### Train Valid split 70:30

In [31]:
import random
random_seed = 42
random.seed(random_seed)
random.shuffle(train_valid_data)

train_valid_split = 0.7
split_index = int(train_valid_split*len(train_valid_data))
print(split_index)

1362


In [32]:
new_train_data = train_valid_data[:split_index]
new_valid_data = train_valid_data[split_index:]

print("Train Data: ", len(new_train_data), new_train_data[:3])
print("Valid Data: ", len(new_valid_data), new_valid_data[:3])

Train Data:  1362 [{'instruction': 'Given the user preference and unpreference, identify whether the user will like the target item by answering "Yes." or "No.".', 'input': '\nUser Preference:"DZT1968(TM)2015 Funny Novelty Summer Beach Outdoor Sun Cap Hat (C)", "LERDU DIY Design Ethnic Lightweight Voile Pattern Scarf, Burgundy Stone Pendant Jewelry Scarf Necklace For Women", "Qiyun (TM) Graduated Butterfly Turquoise Blue Tibet Silver Bib Necklace Earrings Set", \nUser Unpreference:"DZT1968(TM)2015 Funny Novelty Summer Beach Outdoor Sun Cap Hat (C)", "LERDU DIY Design Ethnic Lightweight Voile Pattern Scarf, Burgundy Stone Pendant Jewelry Scarf Necklace For Women", "Qiyun (TM) Graduated Butterfly Turquoise Blue Tibet Silver Bib Necklace Earrings Set", \n Whether the user will like the target movie "Amstt Women Lace Short Sleeve Blouse Casual Fashion Tank Tops T-shirt Medium Green"?', 'output': 'Yes.'}, {'instruction': 'Given the user preference and unpreference, identify whether the user

### Storing as json

In [33]:
import json

with open('./data/train.json', 'w') as train_file:
    json.dump(new_train_data, train_file)

with open('./data/valid.json', 'w') as valid_file:
    json.dump(new_valid_data, valid_file)

with open('./data/test.json', 'w') as test_file:
    json.dump(cleaned_test_data, test_file)

In [35]:
with open('./data/test.json', 'w') as test_file:
    json.dump(cleaned_test_data, test_file)

In [36]:
print(f"Ground Truth: {test_data[4]}")
content = test_data[4]["instruction"] + test_data[4]["input"]
print(content)
print('*'*100)
response = openai.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {
      "role": "user",
      "content": content
    }
  ],
  temperature=0.1,
  max_tokens=64,
  top_p=0.9
)

response = response.choices[0].message.content
print(response)
len(response.split(' '))

Ground Truth: {'instruction': 'Given the user preference and unpreference, identify whether the user will like the target item by answering "Yes." or "No.".', 'input': '\nUser Preference:"NIKE Women\'s Flex Supreme TR 4 Cross Trainer", "NIKE Women\'s Flex Supreme TR 4 Cross Trainer", "NIKE Women\'s Flex Supreme TR 4 Cross Trainer", "NIKE Women\'s Flex Supreme TR 4 Cross Trainer", \n Whether the user will like the target movie "NIKE Women\'s Flex Supreme TR 4 Cross Trainer"?', 'output': 'Yes.'}
Given the user preference and unpreference, identify whether the user will like the target item by answering "Yes." or "No.".
User Preference:"NIKE Women's Flex Supreme TR 4 Cross Trainer", "NIKE Women's Flex Supreme TR 4 Cross Trainer", "NIKE Women's Flex Supreme TR 4 Cross Trainer", "NIKE Women's Flex Supreme TR 4 Cross Trainer", 
 Whether the user will like the target movie "NIKE Women's Flex Supreme TR 4 Cross Trainer"?
*************************************************************************

1